# Scopus

[What are Scopus APIs and how are these used?](https://www.elsevier.com/__data/assets/pdf_file/0007/917179/Scopus-User-Community-Germany-API-final.pdf)

As an abstracting and indexing database, Scopus captures articles being published in virtually all scholarly journals of any significance in the world; and its profiling of authors and institutions makes it easy to find new articles by those authors at those institutions. 

The Scopus UI (user interface) offers many features to that end, allowing librarians, researchers, developers and &business intelligence groups to manually find publications originating from their institution that they can then add to their systems. 

Aside from that user interface, Scopus also has Application Programming Interfaces (APIs)that offer the same features, but then in a machine-readable format that enables software, rather than humans on the UI (User Interface), to find articles, authors and institutions in Scopus.

This allows developers to write programs that automatically extract data from Scopus, and add that data to their systems

### Data Model

The Scopus data model is designed around the notion that **articles** are written by **authors** that are affiliated with **institutions**.


### Dependencias

In [17]:
import os
from pathlib import Path
from os import path
import pandas as pd
import pybliometrics
from pybliometrics.scopus import AffiliationRetrieval
from pybliometrics.scopus import AuthorSearch

pd.set_option('display.max_columns', None)


## Extracción

Se va a utilizar la libreria de [pybliometrics](https://pybliometrics.readthedocs.io/en/stable/index.html). 

Para eso es necesario utilizar una credencial que será almacenada en un archivo de configuración. [Más info](https://pybliometrics.readthedocs.io/en/stable/configuration.html)

In [2]:
os.environ['PYB_CONFIG_FILE'] = "{main_path}/config/scopus_config.ini".format(main_path = Path.cwd().parent.parent.parent)

if (not path.exists(os.environ['PYB_CONFIG_FILE'])):
    # TODO no esta creando el archivo de configuración en la carpeta indicada, sino en ~/.pybliometrics/config.ini
    pybliometrics.scopus.utils.create_config()

Para recuperar datos sobre instituciones de Scopus, se utiliza 

* [Affiliation Retrieval API](https://dev.elsevier.com/documentation/AffiliationRetrievalAPI.wadl) a través de la clase [AffiliationRetrieval](https://pybliometrics.readthedocs.io/en/stable/classes/AffiliationRetrieval.html)
* [Author Search API](https://dev.elsevier.com/documentation/AuthorSearchAPI.wadl) a través de la clase [AuthorSearch](https://pybliometrics.readthedocs.io/en/stable/classes/AuthorSearch.html)
* [Scopus Search API
](https://dev.elsevier.com/documentation/SCOPUSSearchAPI.wadl) a través de la clase [ScopusSearch](https://pybliometrics.readthedocs.io/en/stable/classes/ScopusSearch.html)

El proceso de extracción sera

1. Se consulta los autores de la UNLP a partir del ID de filiación en Scopus, el '60032057'
2. Se almacena los autores en parquet
3. Se obtienen las publicaciones de cada autorpara que a partir de la institución de origen de cada autor poder calcular su filiación
4. Se almacena la filiación de cada autor y las instituciones en parquet

In [25]:
# info de "Universidad Nacional de La Plata" a través del identificador '60032057'
aff = AffiliationRetrieval(60032057)
print(aff)

# recupero autores
author_search = AuthorSearch('AF-ID(60032057)')
df = pd.DataFrame(author_search.authors)

df.to_parquet('scopus_authors_sample.parquet.gzip', compression='gzip')  

df

#ver https://dev.elsevier.com/sc_search_tips.html


Universidad Nacional de La Plata in La Plata in Argentina,
has 3,317 associated author(s) and 15,900 associated document(s) as of 2022-11-01


,eid,orcid,surname,initials,givenname,affiliation,documents,affiliation_id,city,country,areas
0,9-s2.0-56114080200,None,Castro,E.A.,Eduardo Alberto,Universidad Nacional de La Plata,476,60032057,La Plata,Argentina,CHEM (403); PHYS (281); COMP (114)
1,9-s2.0-6602863112,None,Azzaroni,O.,Omar,Universidad Nacional de La Plata,241,60032057,La Plata,Argentina,MATE (213); CHEM (189); PHYS (87)
2,9-s2.0-7005511248,None,Romanelli,G.P.,Gustavo Pablo,Universidad Nacional de La Plata,204,60032057,La Plata,Argentina,CHEM (171); CENG (143); ENVI (52)
3,9-s2.0-7004101620,0000-0003-2771-7805,Althaus,L.G.,L. G.,Consejo Nacional de Investigaciones Científica...,200,60004518,Buenos Aires,Argentina,EART (169); PHYS (159); ENGI (2)
4,9-s2.0-7004599588,0000-0003-2954-6675,Muravchik,C.H.,Carlos Horacio,Comision de Investigaciones Cientificas - La P...,159,60020211,La Plata,Argentina,COMP (127); ENGI (121); MATH (24)
...,...,...,...,...,...,...,...,...,...,...,...
3312,9-s2.0-12344569900,None,Urcola,U.,U.,Universidad Nacional de La Plata,1,60032057,La Plata,Argentina,MATE (2); PHYS (1); ENGI (1)
3313,9-s2.0-12140966300,None,Dela Sota,L.,Luzbel,Universidad Nacional de La Plata,1,60032057,La Plata,Argentina,VETE (1)
3314,9-s2.0-12140525200,None,Legarto,M.L.,María Leticia,Universidad Nacional de La Plata,1,60032057,La Plata,Argentina,CHEM (3); BIOC (1); PHYS (1)
3315,9-s2.0-12042407300,None,Schamun,A.,Alejandro,Universidad Nacional de La Plata,1,60032057,La Plata,Argentina,PSYC (1); MEDI (1)


#### Recuperación de publicaciones

In [24]:
#### Recuperación de publicaciones

#pubs = 

# Se obtienen las publicaciones de cada autorpara que a partir de la institución de origen de cada autor poder calcular su filiación
# Se almacena la filiación de cada autor y las instituciones en parquet

## Transformación

### Carga

In [18]:
df = pd.read_parquet('scopus_authors_sample.parquet.gzip', engine='pyarrow')

# normalizo tipos de datos
df = df.convert_dtypes()
df.dtypes

eid               string
orcid             string
surname           string
initials          string
givenname         string
affiliation       string
documents          Int64
affiliation_id    string
city              string
country           string
areas             string
dtype: object


### Valores Faltantes

94,78 % de autores carece de ORCID, un 0,54 % no registra una ciudad, y aproximadamente 0,15 % no registra ni iniciales ni nombre, mientras que un 0,06 no registra un pais.

In [9]:
na_ratio = ((df.isnull().sum() / len(df))*100).sort_values(ascending = False)
print(na_ratio)

orcid             94.784444
city               0.542659
initials           0.150739
givenname          0.150739
country            0.060295
eid                0.000000
surname            0.000000
affiliation        0.000000
documents          0.000000
affiliation_id     0.000000
areas              0.000000
dtype: float64


### Filiación

Calculo la filiación a partir de las fechas de las publicaciones de cada autor 

In [23]:
author_eid = df.head(1)['eid']
author_eid

0    9-s2.0-56114080200
Name: eid, dtype: string